# Autoreload and logging config

In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

# configure logging
import logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger('Assignment2RRBSDataSource').setLevel(logging.DEBUG)
logging.getLogger('GlmNetEpigeneticClockTrainer').setLevel(logging.DEBUG)
logging.getLogger('download_file').setLevel(logging.DEBUG)

# Download data files

In [6]:
%%time
import os
from epigenetic_clock_assignment import download_file

data_dir = 'data/assignment2'
meta_csv_filepath = os.path.join(data_dir, 'MetaPolished.csv')
features_csv_filepath = os.path.join(data_dir, 'FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv')
features_zip_filepath = os.path.join(data_dir, 'FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv.zip')

# download meta file
download_file(url='https://kerepesi.web.elte.hu/bioinf_ora/MetaPolished.csv',
              out_filepath=meta_csv_filepath)

# download features file if does not exists
download_file(url='https://kerepesi.web.elte.hu/bioinf_ora/FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv.zip',
              out_filepath=features_csv_filepath, 
              intermediate_archive_filepath=features_zip_filepath,
              archive_format='zip')


DEBUG:download_file:file data/assignment2/MetaPolished.csv already exists; do not download again
DEBUG:download_file:file data/assignment2/FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv does not exists; need to download
DEBUG:download_file:creating directory "data/assignment2" if does not exists...
DEBUG:download_file:downloading...


 99% [..................................................................... ] 737869824 / 738850505

DEBUG:download_file:ungzipping data/assignment2/FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv.zip...
DEBUG:download_file:unzipping data/assignment2/FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv.zip...


100% [......................................................................] 738850505 / 738850505

DEBUG:download_file:file data/assignment2/FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv ready


CPU times: user 34.4 s, sys: 16.5 s, total: 50.8 s
Wall time: 3min 39s


# Load data

In [ ]:
%%time

import os
from epigenetic_clock_assignment import Assignment2RRBSDataSource

data_source = Assignment2RRBSDataSource(
    meta_csv_filepath=meta_csv_filepath,
    features_csv_filepath=features_csv_filepath,
    features_pickle_cache_filepath=os.path.join(data_dir, 'features_cache.pickle'))

X, y, meta_df = data_source.load()

print(meta_df)

DEBUG:Assignment2RRBSDataSource:load
DEBUG:Assignment2RRBSDataSource:loading "data/assignment2/FTGenForGenomicCov_MergeStrands.py-cov5.csv-dropna.py-1.0.csv-Format.py.csv" using dask...
DEBUG:Assignment2RRBSDataSource:setting "Pos" field as index...


# Hyperparameter optimization (ElasticNet $\lambda, \alpha$)

In [4]:
%%time
from epigenetic_clock_assignment import GlmNetEpigeneticClockTrainer

clock_trainer = GlmNetEpigeneticClockTrainer(
    n_parallel_jobs=1,
    n_cv_fold=3,
    # the weight of standard error used to determine lambda_best, 0 means use lambda_max (see glmnet.ElasticNet)
    std_error_weight_for_lambda_best=0.5,
    seed=42
)

X_train, X_test, y_train, y_test, meta_train, meta_test = clock_trainer.train_test_split(X, y, meta_df,
                                                                  test_size=0.2)

hyperparameter_result, hyperparameter_stats, best_model = clock_trainer.hyperparameter_optimization(
    X_train=X_train, 
    y_train=y_train, 
    alpha_count= 2 + 1)


NameError: name 'X' is not defined

# Plot hyperparameter optimization result

In [5]:
clock_trainer.plot_hyperparameter_optimization_result(
    hyperparameter_result=hyperparameter_result, 
    hyperparameter_stats=hyperparameter_stats)

NameError: name 'hyperparameter_result' is not defined

# Predict on test data using the best model

In [ ]:
best_lambda = hyperparameter_result.lambd
best_alpha = hyperparameter_result.alpha

# sanity check
assert best_model.alpha == best_alpha
assert best_model.lambda_best_ == best_lambda

# predict
y_test_pred = best_model.predict(X = X_test, lamb=best_lambda)


# Plot linear regression result on test data

In [ ]:
test_stats = clock_trainer.calculate_statistics(y_true=y_test, y_pred=y_test_pred)
clock_trainer.plot_linear_regression_result(
    y_true=y_test, 
    y_pred=y_test_pred, 
    stats=test_stats, 
    alpha=best_alpha, 
    lamb=best_lambda,
    title_prefix='Test set',
    hue=meta_test['Condition'],
    style=meta_test['Gender'])

print(test_stats)


In [ ]:
# predict
y_pred = best_model.predict(X = X, lamb=best_lambda)


In [ ]:
stats = clock_trainer.calculate_statistics(y_true=y, y_pred=y_pred)
clock_trainer.plot_linear_regression_result(
    y_true=y, 
    y_pred=y_pred, 
    stats=test_stats, 
    alpha=best_alpha, 
    lamb=best_lambda,
    title_prefix='All instances',
    hue=meta_df['Condition'],
    style=meta_df['Gender'])



In [ ]:
import seaborn as sns
import scipy.stats as scipystats
import pandas as pd
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator

order = ['Healthy', 'MDD', 'SA']
fig, ax = plt.subplots()

plotting_parameters = {
    'x': meta_df['Condition'],
    'y': stats.age_acceleration,
    'order': order
}

sns.boxplot(ax=ax, **plotting_parameters)

meta_with_stats_df = meta_df.copy()
meta_with_stats_df['age_acceleration'] = stats.age_acceleration
meta_with_stats_df['delta_age'] = stats.delta_age
meta_with_stats_df['age'] = y
meta_with_stats_df['pred_age'] = y_pred
print(meta_with_stats_df)

print(meta_df.groupby('Condition').count())

def check_normality(x: pd.Series):
    res = scipystats.shapiro(x) 
    return res.pvalue

print(meta_with_stats_df['age_acceleration'].agg(is_normal_p_value=lambda x: check_normality(x)))

print(meta_with_stats_df.groupby('Condition')['age_acceleration'].agg(is_normal_p_value=lambda x: check_normality(x)))
# age acceleration is not normally distributed

print('age is normal', meta_with_stats_df.groupby('Condition')['age'].agg(is_normal_p_value=lambda x: check_normality(x)))
print('age is normal in whole dataset', meta_with_stats_df['age'].agg(is_normal_p_value=lambda x: check_normality(x)))



# non normally distributed, we will use Mann-Whitney-Wilcoxon

condition_groupby = meta_with_stats_df.groupby('Condition')
healthy_group = condition_groupby.get_group('Healthy')
mdd_group = condition_groupby.get_group('MDD')
sa_group = condition_groupby.get_group('SA')

healthy_vs_mdd = scipystats.mannwhitneyu(healthy_group['age_acceleration'], mdd_group['age_acceleration'])
healthy_vs_sa = scipystats.mannwhitneyu(healthy_group['age_acceleration'], sa_group['age_acceleration'])
mdd_vs_sa = scipystats.mannwhitneyu(mdd_group['age_acceleration'], sa_group['age_acceleration'])
print('healthy_vs_mdd', healthy_vs_mdd)
print('healthy_vs_sa', healthy_vs_sa)
print('mdd_vs_sa', mdd_vs_sa)
# TODO: https://levelup.gitconnected.com/statistics-on-seaborn-plots-with-statannotations-2bfce0394c00

pairs = [('Healthy', 'MDD'), ('Healthy', 'SA'), ('MDD', 'SA')]

annotator = Annotator(ax, pairs, **plotting_parameters)
#annotator.set_custom_annotations(['1', '2', '3'])
annotator.configure(text_format="simple", test_short_name='Mann-Whitney-Wilcoxon')
annotator.set_pvalues([healthy_vs_mdd.pvalue, healthy_vs_sa.pvalue, mdd_vs_sa.pvalue])
annotator.annotate()
ax.set_ylabel('Age acceleration')
plt.show()

sns.histplot(data=meta_with_stats_df, x='age', hue='Condition', multiple='stack')
plt.show()


In [ ]:
pd.__version__

In [ ]:
from IPython.display import display
# match

#print(healthy_group, type(healthy_group))

#healthy_group.sort_values('age')

healthy_mdd_merged = pd.merge_asof(healthy_group.rename(columns={'age':'age_healthy'}).sort_values('age_healthy'), mdd_group.rename(columns={'age':'age_mdd'}).sort_values('age_mdd'), 
                                   right_on='age_mdd', left_on='age_healthy',
                                   suffixes=('_healthy', '_mdd'), direction='nearest', tolerance=0)

healthy_mdd_merged = healthy_mdd_merged.dropna()
#alma = pd.DataFrame()
#alma['age'] = healthy_mdd_merged['age']
#alma['age_acceleration_healthy'] = healthy_mdd_merged['age_acceleration_healthy']
#healthy_mdd_merged = healthy_mdd_merged.reset_index(drop=True)


healthy_mdd_sa_merged = pd.merge_asof(left=healthy_mdd_merged.sort_values('age_healthy'), right=sa_group.rename(columns={'age':'age_sa', 'delta_age':'delta_age_sa', 'age_acceleration': 'age_acceleration_sa'}) .sort_values('age_sa'), 
                                   right_on='age_sa', left_on='age_healthy',
                                    direction='nearest', tolerance=0)

healthy_mdd_sa_merged = healthy_mdd_sa_merged[['age_healthy', 'age_mdd', 'age_sa',
                                               'delta_age_healthy', 'delta_age_mdd', 'delta_age_sa',
                                              'age_acceleration_healthy', 'age_acceleration_mdd', 'age_acceleration_sa']]
healthy_mdd_sa_merged = healthy_mdd_sa_merged.dropna()
#print('healthy_mdd_merged')
#with pd.option_context('display.max_rows', None, ):
#    display(healthy_mdd_merged)

controlled_dfs = []

class_names = ['Healthy', 'MDD', 'SA']
for i, cls in enumerate(['healthy', 'mdd', 'sa']):
    controlled_df = healthy_mdd_sa_merged[[f'age_{cls}', f'delta_age_{cls}', f'age_acceleration_{cls}']]
    controlled_df = controlled_df.assign(Condition=class_names[i])
    controlled_df = controlled_df.rename(columns={f'age_{cls}': 'age', f'delta_age_{cls}': 'delta_age', f'age_acceleration_{cls}': 'age_acceleration'})
    controlled_dfs.append(controlled_df)
    
merged_controlled_dfs = pd.concat(controlled_dfs)

#controlled_healthy_df = healthy_mdd_sa_merged[['age_healthy', 'delta_age_healthy', 'age_acceleration_healthy']]
#controlled_healthy_df = controlled_healthy_df.assign(Condition='Healthy')

#controlled_mdd_df = healthy_mdd_sa_merged[['age_mdd', 'delta_age_mdd', 'age_acceleration_mdd']]
#controlled_mdd_df = controlled_mdd_df.assign(Condition= 'MDD')

#controlled_sa_df = healthy_mdd_sa_merged[['age_sa', 'delta_age_sa', 'age_acceleration_sa']]
#controlled_sa_df = controlled_sa_df.assign(Condition= 'SA')


print('healthy_mdd_sa_merged')
with pd.option_context('display.max_rows', None, ):
    display(merged_controlled_dfs)

#print(type(healthy_mdd_merged))

order = ['Healthy', 'MDD', 'SA']
fig, ax = plt.subplots()

plotting_parameters = {
    'x': merged_controlled_dfs['Condition'],
    'y': merged_controlled_dfs['age_acceleration'],
    'order': order
}

sns.boxplot(ax=ax, **plotting_parameters)

condition_groupby = merged_controlled_dfs.groupby('Condition')
healthy_group = condition_groupby.get_group('Healthy')
mdd_group = condition_groupby.get_group('MDD')
sa_group = condition_groupby.get_group('SA')

healthy_vs_mdd = scipystats.mannwhitneyu(healthy_group['age_acceleration'], mdd_group['age_acceleration'])
healthy_vs_sa = scipystats.mannwhitneyu(healthy_group['age_acceleration'], sa_group['age_acceleration'])
mdd_vs_sa = scipystats.mannwhitneyu(mdd_group['age_acceleration'], sa_group['age_acceleration'])
print('healthy_vs_mdd', healthy_vs_mdd)
print('healthy_vs_sa', healthy_vs_sa)
print('mdd_vs_sa', mdd_vs_sa)


annotator = Annotator(ax, pairs, **plotting_parameters)
#annotator.set_custom_annotations(['1', '2', '3'])
annotator.configure(text_format="simple", test_short_name='Mann-Whitney-Wilcoxon')
annotator.set_pvalues([healthy_vs_mdd.pvalue, healthy_vs_sa.pvalue, mdd_vs_sa.pvalue])
annotator.annotate()
ax.set_ylabel('Age acceleration')
plt.show()


#healthy_group = condition_groupby.get_group('Healthy')
#mdd_group = condition_groupby.get_group('MDD')
#sa_group = condition_groupby.get_group('SA')

